In [3]:
from pynq import Overlay
from pynq import allocate
import numpy as np
import time
import os

# 1. 系統初始化

overlay = Overlay("example.bit") # 改成自己的bitstream檔名
cdma = overlay.axi_cdma_0
bram_ctrl = overlay.axi_bram_ctrl_0 

# 定義暫存器
CDMACR, CDMASR, SA_REG, DA_REG, BTT_REG = 0x00, 0x04, 0x18, 0x20, 0x28
# 檢查 Address Editor，確認 BRAM 地址
BRAM_PHYS_ADDR = 0x40000000 
# 檢查 BRAM 大小 假設已設為64KB
BRAM_SIZE_LIMIT = 65536 

# 2. 讀取並處理檔案 (File Handling)

FILENAME = "example1.txt"
file_size = os.path.getsize(FILENAME)

print(f"File: {FILENAME}, Size: {file_size} bytes")

if file_size > BRAM_SIZE_LIMIT:
    print(f"Warning: File ({file_size}) is larger than BRAM ({BRAM_SIZE_LIMIT}). Truncating data!")
    transfer_size = BRAM_SIZE_LIMIT
else:
    transfer_size = file_size

# 讀取檔案內容為二進位
with open(FILENAME, 'rb') as f:
    file_content = f.read(transfer_size)

# 轉換為 numpy array (uint8 為單位，因為是文字/bytes)
# 雖然 CDMA 建議對齊 32-bit，但用 uint8 比較好處理任意長度文字
data_buffer = allocate(shape=(transfer_size,), dtype=np.uint8)

# 將檔案內容填入 buffer
for i, byte in enumerate(file_content):
    data_buffer[i] = byte

# Flush Cache 到 DDR
data_buffer.flush()
print(f"Buffer populated with {transfer_size} bytes and flushed to DDR.")

# 3. 執行 CDMA 大量傳輸

def run_large_transfer(src, dst, length):
    # Reset
    cdma.write(CDMACR, 0x04)
    while cdma.read(CDMACR) & 0x04: pass
    
    # Setup
    cdma.write(SA_REG, src)
    cdma.write(DA_REG, dst)
    
    # Trigger
    start_time = time.time()
    cdma.write(BTT_REG, length)
    
    # Wait
    while not (cdma.read(CDMASR) & 0x02): pass
    end_time = time.time()
    
    # Check Error
    status = cdma.read(CDMASR)
    if status & 0x70:
        raise Exception(f"CDMA Error: {hex(status)}")
    
    return end_time - start_time

print("Starting CDMA Transfer...")
try:
    duration = run_large_transfer(data_buffer.device_address, BRAM_PHYS_ADDR, transfer_size)
    print(f"Transfer Complete! Time taken: {duration:.6f} seconds")
    # 計算頻寬 (MB/s)
    bandwidth = (transfer_size / 1024 / 1024) / duration
    print(f"Effective Bandwidth: {bandwidth:.2f} MB/s")
    
except Exception as e:
    print(e)

# 4. 驗證 (CPU 讀取前 100 bytes)


print("\nVerifying first 100 characters from BRAM...")

# 讀取回來 (注意: PYNQ MMIO 讀取通常是 4 bytes aligned)
# 這裡我們簡單示範讀取前幾個 Word
verified_text = ""
for i in range(0, 100, 4): 
    # 每次讀 4 bytes
    word = bram_ctrl.read(i)
    # 將 int 轉回 4個 chars (Little Endian)
    bytes_val = word.to_bytes(4, byteorder='little')
    try:
        verified_text += bytes_val.decode('utf-8')
    except:
        verified_text += "????"

print(f"--- BRAM Content Start ---\n{verified_text}\n--- BRAM Content End ---")

File: example1.txt, Size: 11431186 bytes
Buffer populated with 65536 bytes and flushed to DDR.
Starting CDMA Transfer...
Transfer Complete! Time taken: 0.000235 seconds
Effective Bandwidth: 266.14 MB/s

Verifying first 100 characters from BRAM...
--- BRAM Content Start ---
Input INPUT1 1253995 0
Input INPUT2 1296040 1140825
Input INPUT3 1296040 1187185
Input INPUT4 576075
--- BRAM Content End ---
